In [2]:
import logging
import os
from typing import Annotated, List, Sequence, TypedDict

import tools as tools_
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, SystemMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import logging
import os
from typing import Annotated, List, Sequence, TypedDict

import tools as tools_
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, SystemMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, PointStruct, VectorParams
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from qdrant_client.models import FieldCondition, Filter, MatchValue


d:\Studia\AquaRAG\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3670: UserWarning: Parameters {'temperature'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
import logging
import os
import uuid
from typing import List

from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_openai import ChatOpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import FieldCondition, Filter, MatchValue

load_dotenv()
collection_name = os.environ["COLLECTION_NAME"]
collection_name_memories = os.environ["COLLECTION_MEMORIES"]
model = ChatOpenAI(model=os.environ["LLM"], temperature=0)


def vectorize_user_query(query: str) -> List[float]:
    """Embeds user_query vector"""
    embedder = NVIDIAEmbeddings(model=os.environ["EMBEDDER"])
    return embedder.embed_query(query)



def retrive_data_from_db(user_query: str) -> str:
    """Using user query input returns data from vector database as a string of anserws from document"""

client = QdrantClient(url=os.getenv("QDRANT_ADDRESS"))
response = []

points = client.query_points(
	collection_name=collection_name,
	query=vectorize_user_query(query="Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?"),
	limit=5,
	with_payload=True,
    with_vectors=True
).points

response.extend(points)

client.close()


In [36]:
response

[ScoredPoint(id=947, version=947, score=0.7059065, payload={'content': {'metadata': {'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-24T10:08:08+01:00', 'title': 'Akt prawny', 'author': 'Władysław Baksza', 'moddate': '2024-01-24T10:08:08+01:00', 'source': './data/prawod_wodne.pdf', 'total_pages': 444}, 'page_content': 'spraw gospodarki wodnej, na podstawie art. 190 ust. 9.\n\x0c©Kancelaria Sejmu    s. 155/444 \n \n2024-01-24 \n \n13. Jeżeli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym \nmowa w ust. 1, lub nie uzyskał decyzji o legalizacji urządzenia wodnego, organ \nwłaściwy w sprawach pozwoleń wodnoprawnych na wykonywanie urzą dzeń'}}, vector=[-0.014605522, 0.008836569, 0.01004225, -0.00626496, -0.029302614, -0.019290887, -0.01947403, -0.004677735, 0.0037124276, 0.012171268, -9.604189e-05, 0.01689479, 0.014361333, -0.0038440605, -0.0027013347, -0.021076515, -0.0049906014, 0.000254204, 0.024525678, 0.007936125, 0.00807

In [27]:
new_list = []
for i in response:
    new_list.append(i.payload['content']['page_content'])

In [28]:
new_list

['spraw gospodarki wodnej, na podstawie art. 190 ust. 9.\n\x0c©Kancelaria Sejmu    s. 155/444 \n \n2024-01-24 \n \n13. Jeżeli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym \nmowa w ust. 1, lub nie uzyskał decyzji o legalizacji urządzenia wodnego, organ \nwłaściwy w sprawach pozwoleń wodnoprawnych na wykonywanie urzą dzeń',
 'Art. 190. 1. Jeżeli urządzenie wodne zostało wykonane bez wymaganego \npozwolenia wodnoprawnego lub zgłoszenia,  właściciel tego urządzenia może \nwystąpić z  wnioskiem o  jego legalizacj ę, do którego dołącza odpowiednio \ndokumenty, o których mowa w art. 407 ust. 2 oraz w art. 422. \n2. Organ właściwy w sprawach pozwoleń wodnoprawnych na wykonywanie',
 'wodnego, o którym mowa w ust. 1, lub ograniczeniu korzystania z  tego \nurządzenia odpowiednio właściciela urządzenia wodnego lub podmiot \nkorzystający z tego urządzenia – jeżeli jest możliwe ich ustalenie, \nb) podejmuje działania w  celu wszczęcia procedury legalizacji urządzenia \nwodnego ‒ 

In [29]:
from sentence_transformers import CrossEncoder

# Load a pre-trained CrossEncoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Predict scores for a pair of sentences
 
scores = model.predict([
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[0]),
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[1]),
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[2]),
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[3]),
    ("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[4]),
])

# => ar

d:\Studia\AquaRAG\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [37]:
from sentence_transformers.cross_encoder import CrossEncoder

model_path = "radlab/polish-cross-encoder"
model = CrossEncoder(model_path)


questions = ["Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?"] * 5
answers = new_list
for question in questions:
    context_with_question = [(s, question) for s in answers]
    results = sorted(
        {
            idx: r for idx, r in enumerate(model.predict(context_with_question))
        }.items(),
        key=lambda x: x[1],
        reverse=True,
    )

    print(f"QUESTION: {question}")
    print("ANSWERS (sorted):")
    for idx, score in results:
        print(f"\t[{score}]\t{idx}")
    print("")

d:\Studia\AquaRAG\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


QUESTION: Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?
ANSWERS (sorted):
	[0.6784870624542236]	0
	[0.6588813066482544]	2
	[0.6263067722320557]	4
	[0.5765359401702881]	1
	[0.5701243877410889]	3

QUESTION: Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?
ANSWERS (sorted):
	[0.6784870624542236]	0
	[0.6588813066482544]	2
	[0.6263067722320557]	4
	[0.5765359401702881]	1
	[0.5701243877410889]	3

QUESTION: Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?
ANSWERS (sorted):
	[0.6784870624542236]	0
	[0.6588813066482544]	2
	[0.6263067722320557]	4
	[0.5765359401702881]	1
	[0.5701243877410889]	3

QUESTION: Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?
ANSWERS (sorted):
	[0.6784870624542236]	0
	[0.6588813066482544]	2
	[0.6263067722320557]	4
	[0.5765359401702881]	1
	[0.5701243877410889]	3

QUES